In [1]:
from mgo_model import MgoModel
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [ ]:
model = MgoModel('stats')

villages = ['Luchelegwa', 'Mkahara', 'Luchelegwa']
coords = {'Luchelegwa': [-10.4021, 38.961], 'Mkahara': [-10.7257, 39.9911], 'Luchelegwa': [-10.4021, 38.9614]}

min_area = 50
demands = [5, 10, 25, 50, 100]
tariffs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]
gen_cost = 1500
cost_wire = 5
cost_conn = 80
opex_ratio = 1
years = 10
discount_rate = 10
max_length = 10000

dfs = []
for village in villages:
    print(village)
    model.set_village(village)
    model.map_with_pv(coords[village][0], coords[village][1])
    
    sensitivity = []
    for demand in demands:
        print('-----', demand, '------')
        for tariff in tariffs:
            new_map, results, village, zipped = model.run_model(min_area, demand, tariff, gen_cost, cost_wire,
                                                                cost_conn, opex_ratio, years, discount_rate, max_length)
            npv = results['npv']
            print(tariff, '--', npv)
            sensitivity.append({'demand': demand, 'tariff': tariff, 'npv': npv})
    
    df = pd.DataFrame(sensitivity)
    dfs.append(df)

In [3]:
p = figure(title = "Click legend entries to hide/show")
p.xaxis.axis_label = 'Tariff (USD/kW)'
p.yaxis.axis_label = 'NPV (\'000 USD)'

colormap = {5: '#006d2c', 10: '#2ca25f', 25: '#66c2a4', 50: '#b2e2e2', 100: '#edf8fb'}
p.circle(dfs[0]["tariff"],
         dfs[0]["npv"]/1000,
         color=[colormap[x] for x in dfs[0]['demand']],
         fill_alpha=0.5,
         size=10,
         legend='Luchelegwa')

colormap = {5: '#b30000', 10: '#e34a33', 25: '#fc8d59', 50: '#fdcc8a', 100: '#fef0d9'}
p.circle(dfs[1]["tariff"],
         dfs[1]["npv"]/1000,
         color=[colormap[x] for x in dfs[1]['demand']],
         fill_alpha=0.5,
         size=10,
         legend='Mkahara')

colormap = {5: '#810f7c', 10: '#8856a7', 25: '#8c96c6', 50: '#b3cde3', 100: '#edf8fb'} 
p.circle(dfs[2]["tariff"],
         dfs[2]["npv"]/1000,
         color=[colormap[x] for x in dfs[2]['demand']],
         fill_alpha=0.5,
         size=10,
         legend='Nakiu')

p.circle(dfs[0]["tariff"], dfs[0]["npv"]/1000, fill_alpha=0, line_alpha=0,
         legend='Demand levels (kWh/pp/month): 5, 10, 25, 50, 100')
p.circle(dfs[0]["tariff"], dfs[0]["npv"]/1000, fill_alpha=0, line_alpha=0,
         legend='Lighter colours = higher demand')

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("sensitivity.html", title="TZ village sensitivity")
show(p)